# motive
1. Be comfortable with both vector and graph embeding
2. store and retrive from neo4j
## notebook work plan

Vector embeding
1. prepare env and define metadata extractor target
2. load document
3. embed with opensource HF model (vector embeding)
4. store into neo4j (vector store)
5. build query agent

KG part
1. use metadata kg option true
2. store knowledege graph embeding to neo4j
3. build kg query agent

In [ ]:
# !pip install -q pygpt4all
!pip install -q llama-index
!pip install -q sentence_transformers
!pip install -q transformers
!pip install -q einops
!pip install -q accelerate
!pip install -q pypdf
!pip install -q langchain
!pip install -q llama-cpp-python
!pip install -q neo4j

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from IPython.display import Markdown, display
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from llama_index.node_parser.simple import SimpleNodeParser
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from llama_index import (
    GPTVectorStoreIndex,
    LangchainEmbedding,
    LLMPredictor,
    ServiceContext,
    StorageContext,
    download_loader,
    PromptHelper,
     SimpleDirectoryReader,
    VectorStoreIndex,
     SummaryIndex,
    KnowledgeGraphIndex,
    SimpleKeywordTableIndex,
    ServiceContext,
)
from llama_index.schema import IndexNode
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.llms import OpenAI
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from langchain.llms import CTransformers
from langchain.memory import ConversationBufferMemory
import os

In [ ]:
# using HF embedding
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [ ]:
# using mistral-7b
# using llama2-13b

llm3 = CTransformers(model="../company_QA/llama-2-13b-chat.Q4_0.gguf",
                     config={'max_new_tokens': 256,
                             'temperature': 0.2,
                             'context_length':3900})
# llm3 = CTransformers(model="mistral-7b-instruct-v0.1.Q4_K_M.gguf", config={'max_new_tokens': 556, 'temperature': 0.2, 'context_length':3900})

In [ ]:

from llama_index import load_index_from_storage, StorageContext
from llama_index.node_parser.extractors import (
    MetadataExtractor,
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    EntityExtractor,
)

from llama_index.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(separator=" ", chunk_size=1512, chunk_overlap=128)

metadata_extractor = MetadataExtractor(
    extractors=[
        TitleExtractor(nodes=2,llm=llm3),
        # QuestionsAnsweredExtractor(questions=3),
        #  SummaryExtractor(summaries=["prev", "self"],llm=llm),
        KeywordExtractor(keywords=5,llm=llm3),
        # EntityExtractor(prediction_threshold=0.5),
    ],
)

node_parser = SimpleNodeParser.from_defaults(
    text_splitter=text_splitter,
    metadata_extractor=metadata_extractor,
)

service_context = ServiceContext.from_defaults(embed_model=embed_model,
                                               llm=llm3,
                                                node_parser=node_parser)


In [ ]:
from llama_index import (
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext,
    KnowledgeGraphIndex,
)
from llama_index.graph_stores import SimpleGraphStore

from llama_index.llms import OpenAI
from IPython.display import Markdown, display
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader

# load documents
# documents = SimpleDirectoryReader("./data_wt").load_data()
# for the pdf file
loader = DirectoryLoader('./data_wt', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()
len(documents)

18

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
# Chunk the document
documents = text_splitter.split_documents(documents)
len(documents)

21

# neo4j vector embedding

In [ ]:
from langchain.vectorstores import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings

# Neo4j Aura credentials
url="neo4j+s://6b65.databases.neo4j.io"
username="neo4j"
password="5JLKyskCjMPBeME2dw"

# Instantiate Neo4j vector from documents
# neo4j_vector = Neo4jVector.from_documents(
#     documents,
#     embed_model,
#     url=url,
#     username=username,
#     password=password
# )

# similarity search

In [ ]:
query = "Tell me more about Intel and amd future plans , who is doing best??"

results = neo4j_vector.similarity_search(query, k=1)
print(results[0].page_content)

Based on the information provided, it appears that AMD experienced a significant increase in market
share during the pandemic, but has since slightly decreased in 2022. The company's profit and loss
income for 2023 is not yet available. Strategic plans for the future include continued development of
new CPU technologies and increased competition with Intel. Current challenges include the ongoing
impact of the pandemic on the technology industry and the increasing popularity of cloud computing.
Opportunities for the company include the growing demand for high-performance CPUs in gaming,
content creation, and other applications.


In [ ]:
query = "future between riskv and arm, where should I invest?"

results = neo4j_vector.similarity_search(query, k=2)
print(results[0].page_content)

Based on the information provided, it appears that Risk-V architecture company is a relatively new and
innovative company that is making significant strides in the field of CPU architecture. The company has
received investment from several major chipmakers, including Google and Qualcomm, and is working
to develop reference architectures based on the RISC-V standard. This could potentially challenge the
near-ubiquitous technology of Arm, which is currently the dominant player in the market. Additionally,
the company is working on developing low-power, custom CPUs for use with Wear OS by Google,
which could lead to more efficient and powerful wearable devices. Overall, it seems that Risk-V
architecture company is a promising new player in the tech industry, with a focus on innovation and
collaboration.


In [ ]:
query = "which company investing on research more frequently? also tell me their profit pattern"

results = neo4j_vector.similarity_search(query, k=3)
print(results[0].page_content)

company has a strong track record of financial performance and is well-positioned for


In [ ]:
query = "what is new doing by amd gpu division in year 2023"

results = neo4j_vector.similarity_search(query, k=3)
print(results[0].page_content)

Based on the information provided, it appears that AMD GPU company is a well-established and highly
successful technology company that specializes in graphics processing units (GPUs) and other related
technologies. The company has a long history dating back to 1969, and has since become one of the
largest and most pervasive GPU architectures in history, with over 250 billion chips shipped to date.
AMD's GPU architecture is present in 99% of gaming PCs in the world, and the company is constantly
working on developing new and innovative technologies, such as the Radeon Instinct. Additionally, the
company has a strong focus on collaboration and partnerships, with a global network of offices and a
talented engineering team. Overall, it seems that AMD GPU company is a highly successful and
innovative technology company with a strong track record and a bright future.


In [ ]:
query =  """tell me about every company from your database how can we identify capabiltiies that might have a different market
how do we identify process break. companies without revenue or head count / employees
how do we identify capabilities to be added manually back into the taxonomy"""

results = neo4j_vector.similarity_search(query, k=3)
print(results[0].page_content)

Here is the information you requested about Intel CPU company for the points you mentioned: 1. What
makes this company stand out from other companies (key capabilities)? Intel is known for its advanced
performance hybrid architecture, which allows for faster compute performance without compromising
user workflow. The company also has a strong history of innovation, having developed the first
commercial microprocessor in 1971 and continuing to push the boundaries of technology with each
new generation of processors. 2. What research or new things are there that the company is involved
with? Intel is constantly working on new technologies and advancements, such as its recent
achievement of hitting 6GHz with its i9-13900KS processor. The company is also investing in emerging
technologies like artificial intelligence and autonomous driving. 3. What are their profit, loss income,
strategic plans? Intel reported a net income of $22.5 billion in 2022, with revenue of $72.1 billion. The


In [ ]:
query = "compare between amd gpu and risc five architecture "

results = neo4j_vector.similarity_search(query, k=3)
print(results[0].page_content)

Based on the information provided, it appears that AMD GPU company is a well-established and highly
successful technology company that specializes in graphics processing units (GPUs) and other related
technologies. The company has a long history dating back to 1969, and has since become one of the
largest and most pervasive GPU architectures in history, with over 250 billion chips shipped to date.
AMD's GPU architecture is present in 99% of gaming PCs in the world, and the company is constantly
working on developing new and innovative technologies, such as the Radeon Instinct. Additionally, the
company has a strong focus on collaboration and partnerships, with a global network of offices and a
talented engineering team. Overall, it seems that AMD GPU company is a highly successful and
innovative technology company with a strong track record and a bright future.


In [ ]:
query = "what is risk five?"

results = neo4j_vector.similarity_search(query, k=3)
print(results[0].page_content)

Based on the information provided, it appears that Risk-V architecture company is a relatively new and
innovative company that is making significant strides in the field of CPU architecture. The company has
received investment from several major chipmakers, including Google and Qualcomm, and is working
to develop reference architectures based on the RISC-V standard. This could potentially challenge the
near-ubiquitous technology of Arm, which is currently the dominant player in the market. Additionally,
the company is working on developing low-power, custom CPUs for use with Wear OS by Google,
which could lead to more efficient and powerful wearable devices. Overall, it seems that Risk-V
architecture company is a promising new player in the tech industry, with a focus on innovation and
collaboration.


# building chain with Retrieval QA

In [ ]:
from langchain.chains import RetrievalQA

# create the chain to answer questions
chain_QA = RetrievalQA.from_chain_type(llm=llm3,
                                  chain_type="stuff",
                                  retriever=neo4j_vector.as_retriever(),
                                  return_source_documents=True)

In [ ]:
chain_QA(query)

{'query': 'what is risk five?',
 'result': ' Based on the information provided, it appears that Risk-V architecture company is a relatively new and innovative company that is making significant strides in the field of CPU architecture. The company has received investment from several major chipmakers, including Google and Qualcomm, and is working to develop reference architectures based on the RISC-V standard. This could potentially challenge the near-ubiquitous technology of Arm, which is currently the dominant player in the market. Additionally, the company is working on developing low-power, custom CPUs for use with Wear OS by Google, which could lead to more efficient and powerful wearable devices. Overall, it seems that Risk-V architecture company is a promising new player in the tech industry, with a focus on innovation and collaboration.',
 'source_documents': [Document(page_content='Based on the information provided, it appears that Risk-V architecture company is a relatively n

In [ ]:
chain_QA("Tell me more about Intel and amd future plans , who is doing best?")

{'query': 'Tell me more about Intel and amd future plans , who is doing best?',
 'result': ' Based on the information provided, it appears that AMD has been experiencing significant growth in recent years, particularly during the COVID-19 pandemic. The company has a strong focus on research and development, and they have been investing heavily in new technologies such as 6nm and 4nm process nodes. Additionally, AMD has a diverse product lineup that includes both desktop and server CPUs. In terms of financial performance, the company has reported strong revenue and profit growth in recent years, with a market share of around 39% in the desktop CPU market. However, it is worth noting that the company\'s financial performance can be affected by various factors such as global economic conditions and industry trends.\n\nAs for Intel, the company has recently launched its 14th Gen Core desktop processors with up to 24 cores and a 6GHz boost frequency, setting records for desktop processing p

In [ ]:
#resoning
chain_QA("future between riskv and arm, where should I invest?")

{'query': 'future between riskv and arm, where should I invest?',
 'result': " Based on the information provided, both Risk-V and ARM are promising companies in the tech industry, with a focus on innovation and collaboration. However, it's important to note that Risk-V is a relatively new company, while ARM has a long history of success and a strong track record. Additionally, ARM has a much larger market share and presence in the industry, which could potentially make it a more stable investment. Ultimately, the decision on where to invest should be based on your individual financial goals and risk tolerance. It may be helpful to conduct further research and consult with a financial advisor before making a decision.",
 'source_documents': [Document(page_content='Based on the information provided, it appears that Risk-V architecture company is a relatively new and\ninnovative company that is making significant strides in the field of CPU architecture. The company has\nreceived investme

In [ ]:
#11.14-11.25 =10 min
chain_QA("what is new doing by amd gpu division in year 2023")

{'query': 'what is new doing by amd gpu division in year 2023',
 'result': " Based on the information provided, it appears that AMD GPU division is continuing to develop new and innovative technologies, such as the Radeon Instinct, and is focused on increasing competition with Intel. The company's strategic plans for the future include continued development of new CPU technologies and increased competition with Intel. However, without more specific information about AMD's GPU division in 2023, it is difficult to provide a more detailed answer.",
 'source_documents': [Document(page_content="Based on the information provided, it appears that AMD GPU company is a well-established and highly\nsuccessful technology company that specializes in graphics processing units (GPUs) and other related\ntechnologies. The company has a long history dating back to 1969, and has since become one of the\nlargest and most pervasive GPU architectures in history, with over 250 billion chips shipped to date.

In [ ]:
#irrelavent question 10mins
chain_QA( """tell me about every company from your database how can we identify capabiltiies that might have a different market
how do we identify process break. companies without revenue or head count / employees
how do we identify capabilities to be added manually back into the taxonomy""")

{'query': 'tell me about every company from your database how can we identify capabiltiies that might have a different market\nhow do we identify process break. companies without revenue or head count / employees\nhow do we identify capabilities to be added manually back into the taxonomy',
 'result': "\n\nTo identify capabilities that may have a different market, you can look at the company's product offerings and see if they have any products or services that are not currently listed in the taxonomy. For example, if a company has a product that is specifically designed for a niche market, it may be worth adding that capability to the taxonomy as a separate entry. Additionally, you can look at the company's financial statements and see if they have any revenue or profit from these products or services, as this could indicate that there is a market for them.\n\nTo identify process breaks, you can look at the company's organizational structure and see if there are any gaps or overlaps i

# knowledge graph index building

In [ ]:
# load documents
documents_kg = SimpleDirectoryReader("./data_wt").load_data()
len(documents_kg)

18

In [ ]:
from llama_index.storage.storage_context import StorageContext

graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
# index_kg = KnowledgeGraphIndex.from_documents(
#     documents_kg,
#     # max_triplets_per_chunk=2,
#     storage_context=storage_context,
#     service_context=service_context,
# )

In [ ]:
kg_engine = index_kg.as_query_engine(include_text=False,response_mode="tree_summarize")
response = kg_engine.query(
    "Tell me more about Intel and their future plan in 2090",
)
print(response)

In [ ]:
kg_engine = index_kg.as_query_engine(response_mode="tree_summarize")
response = kg_engine.query(
    "which company investing on research more frequently? also tell me their profit pattern",
)
print(response)

# metadata extraction

In [ ]:
nodes = node_parser.get_nodes_from_documents(documents_kg)

In [ ]:
# NOTE: can take a while! around 3-4hr
index_kg_nodes = KnowledgeGraphIndex(
    nodes,
    # max_triplets_per_chunk=2,
    storage_context=storage_context,
    service_context=service_context,
)

(Risk-V Architecture Company, is, a promising new player)
(RISC-V standard, is, a reference architecture)
(Google, invested in, Risk-V Architecture Company)
(Qualcomm, invested in, Risk-V Architecture Company)
(Wear OS by Google, is, a wearable device operating system)
(low-power CPUs, are, custom CPUs for wearable devices)
(Risk-V Architecture Company, is, a promising new player)
(RISC-V, is, a standard for CPU architecture)
(Google, invested in, Risk-V Architecture Company)
(Qualcomm, invested in, Risk-V Architecture Company)
(wearable devices, use, low-power custom CPUs)
(Wear OS by Google, is, a platform for wearable devices)
(Risk-V Architecture Company, is, a promising new player)
(RISC-V standard, is, a reference architecture)
(Google, invested in, Risk-V Architecture Company)
(Qualcomm, invested in, Risk-V Architecture Company)
(Wear OS by Google, is, a wearable device operating system)
(low-power CPUs, are, custom CPUs for wearable devices)
---------------------
Please provide

In [ ]:
kg_engine_node = index_kg_nodes.as_query_engine(response_mode="tree_summarize")
response = kg_engine_node.query(
    "which company investing on research more frequently? also tell me their profit pattern",
)
print(response)



Note: Please provide a detailed answer with proper reasoning and evidence to support your answer.


In [ ]:
nodes

[TextNode(id_='fe483df2-20e6-40fe-84e5-03ab0a28e262', embedding=None, metadata={'page_label': '1', 'file_name': 'Risk-V architecture_2019.pdf', 'document_title': 'Risk-V Architecture Company: A Promising New Player in the Tech Industry.\n\nThe comprehensive title for this document is "Risk-V Architecture Company: A Promising New Player in the Tech Industry". This title effectively captures the main topic of the document, which is an overview of Risk-V Architecture Company and its potential in the tech industry. The title accurately reflects the content of the document, providing readers with a clear understanding of what they can expect to learn from the article.', 'excerpt_keywords': 'based on the information provided, it appears that risk-v architecture company is a relatively new and innovative company that is making significant strides in the field of CPU architecture. the company has received investment from several major chipmakers, including google and qualcomm, and is working t

In [ ]:
len(nodes)

18

In [ ]:
from llama_index.graph_stores import Neo4jGraphStore

graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database="neo4j",
)

storage_context1 = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index = KnowledgeGraphIndex(
    nodes,
    storage_context=storage_context1,
    # max_triplets_per_chunk=2,
    service_context=service_context,
)

(Risk-V Architecture Company, is, a promising new player)
(RISC-V standard, is, a reference architecture)
(Google, invested in, Risk-V Architecture Company)
(Qualcomm, invested in, Risk-V Architecture Company)
(Wear OS by Google, is, a wearable device operating system)
(low-power CPUs, are, custom CPUs for wearable devices)
(Risk-V Architecture Company, is, a promising new player)
(RISC-V, is, CPU architecture standard)
(Google, invested in, Risk-V Architecture Company)
(Qualcomm, invested in, Risk-V Architecture Company)
(Wear OS by Google, is, a wearable device operating system)
(custom CPUs, are, designed for specific applications)
(Risk-V Architecture Company, is, a promising new player)
(RISC-V standard, is, a reference architecture)
(Google, invested in, Risk-V Architecture Company)
(Qualcomm, invested in, Risk-V Architecture Company)
(Wear OS by Google, is, a wearable device operating system)
(low-power CPUs, are, custom CPUs for wearable devices)
(AMD, has made significant stri

Failed to write data to connection ResolvedIPv4Address(('34.126.64.110', 7687)) (ResolvedIPv4Address(('34.126.64.110', 7687)))
Failed to write data to connection IPv4Address(('6a399b65.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.64.110', 7687)))


In [ ]:
kg_e_node = index.as_query_engine(response_mode="tree_summarize")
response = kg_e_node.query(
    "which company investing on research more frequently? also tell me their profit pattern",
)
print(response)



Hint: Analyze the given information carefully to find the answer.

Note: Please provide your answer in a structured format like tables or lists.


In [ ]:
from llama_index.vector_stores import Neo4jVectorStore

vec_store = Neo4jVectorStore(
    username=username,
    password=password,
    url=url,
    database="neo4j",
    embedding_dimension=256,
)

storage_context2 = StorageContext.from_defaults(vector_store=vec_store)

# NOTE: can take a while!
index = VectorStoreIndex(
    nodes,
    storage_context=storage_context2,
    # max_triplets_per_chunk=2,
    service_context=service_context,
)

In [ ]:
kg_e_vec = index.as_query_engine(response_mode="tree_summarize")
response = kg_e_vec.query(
    "which company investing on research more frequently? also tell me their profit pattern",
)
print(response)

 Based on the information provided by both sources, it appears that NVIDIA is investing more frequently in research and development compared to Intel. This can be seen from the excerpts provided, where NVIDIA is mentioned to be actively involved in various research projects such as artificial intelligence, machine learning, and autonomous driving, while Intel's recent achievement is hitting 6GHz with its i9-13900KS processor.

Regarding profit patterns, both companies have reported strong financial performance in their respective years. However, NVIDIA has shown a more consistent pattern of growth, with an 843 percent year-over-year increase in profitability, as reported in its latest financial report. On the other hand, Intel's net income has fluctuated more significantly over the past few years, with a peak of $22.5 billion in 2022 but also experiencing a decline in revenue and profitability in previous years.

Therefore, based on the information provided, it appears that NVIDIA is i

In [ ]:
# !jupyter nbconvert company_QA_neo4j.ipynb --to python